# Data Preprocessing

### Import Data

In [150]:
import pandas as pd
import re

In [151]:
df = pd.read_csv('data/DAGs.csv')
df.head()

,Author,Exposure,Outcome,Direction,Strength,ID,Status
0,NaN,Diabetes,Ischemic stroke,Increase,6.0,1,Final
1,NaN,age,Ischemic stroke,Increase,6.0,1,Final
2,NaN,age,Diabetes,Increase,5.0,1,Final
3,NaN,Sex (Male),Ischemic stroke,Increase,6.0,1,Final
4,NaN,Hypertension,Ischemic stroke,Increase,6.0,1,Final


### Getting a List of Terms

In [152]:
terms = pd.concat([df.Exposure, df.Outcome], ignore_index=True).unique()
pd.DataFrame(terms, columns=['terms']).to_csv('terms.csv', index=False)

## Standardizing Terms with SNOMED CT, LOINC, ATC
* Preferred is SNOMED CT, used LOINC or ATC when SNOMED term was not available

### Merge mapped codes for terms (done manually in Excel) with OHDSI ATHENA database

Load mapped terms
* `term` - Original term name
* `concept_code` - SNOMED, LOINC, or ATC code for term
* `concept_code_2` - For concepts that don't exist on their own that are a combination of two codes (e.g. neuron survival -> neuron + survival rate)
* `time` - If a concept was used multiple times to keep track of its position in the DAG
* `direction` - Whether the direction in the DAG needs to be switched (e.g. female is coded as male so the direction needs to be changed from increase to decrease or vice versa in the original DAG) 

In [153]:
mapped_terms = pd.read_excel('terms_mapped_to_codes.xlsx', sheet_name='terms', dtype={'concept_code': str, 'concept_code_2':str})
mapped_terms.head()

,term,concept_code,concept_code_2,time,direction
0,Diabetes,73211009,NaN,NaN,NaN
1,age,71395006,NaN,NaN,NaN
2,Sex (Male),248153007,NaN,NaN,NaN
3,Hypertension,59621000,NaN,NaN,NaN
4,BMI,60621009,NaN,NaN,NaN


Load Athena database

In [154]:
athena_concepts = pd.read_csv('ohdsi_athena_vocab/CONCEPT.csv', sep='\t', dtype={'concept_code': str}, low_memory=False)
concept_columns = ['concept_id','concept_name','domain_id','vocabulary_id','concept_class_id','concept_code']
athena_concepts = athena_concepts[concept_columns]
athena_concepts.head()

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,concept_code
0,3541502,Adverse reaction to drug primarily affecting t...,Condition,SNOMED,Disorder,694331000000106
1,3542356,Adverse reaction to other central nervous syst...,Condition,SNOMED,Disorder,705311000000105
2,4327638,Borderline,Observation,SNOMED,Qualifier Value,75189007
3,42538812,Somatic hallucination,Condition,SNOMED,Clinical Finding,762620006
4,40629514,Stillbirth,Condition,SNOMED,Clinical Finding,76358005


Merge

In [155]:
primary_concepts = mapped_terms.merge(athena_concepts, how='left', on='concept_code')

In [156]:
secondary_concepts = mapped_terms.merge(athena_concepts, how='left', left_on='concept_code_2', right_on='concept_code', suffixes=('','2'))
secondary_concepts = secondary_concepts[concept_columns].drop('concept_code',axis=1)

In [157]:
all_terms = primary_concepts.merge(secondary_concepts, left_index=True, right_index=True, suffixes=('','_2'))
all_terms.head()

,term,concept_code,concept_code_2,time,direction,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,concept_id_2,concept_name_2,domain_id_2,vocabulary_id_2,concept_class_id_2
0,Diabetes,73211009,NaN,NaN,NaN,201820.0,Diabetes mellitus,Condition,SNOMED,Disorder,NaN,NaN,NaN,NaN,NaN
1,age,71395006,NaN,NaN,NaN,4322978.0,Age factor,Observation,SNOMED,Observable Entity,NaN,NaN,NaN,NaN,NaN
2,Sex (Male),248153007,NaN,NaN,NaN,442985.0,Male,Observation,SNOMED,Clinical Finding,NaN,NaN,NaN,NaN,NaN
3,Hypertension,59621000,NaN,NaN,NaN,320128.0,Essential hypertension,Condition,SNOMED,Disorder,NaN,NaN,NaN,NaN,NaN
4,BMI,60621009,NaN,NaN,NaN,4245997.0,Body mass index,Measurement,SNOMED,Observable Entity,NaN,NaN,NaN,NaN,NaN


### Clean and Export Check List of Terms

In [158]:
terms_sorted = all_terms.sort_values('concept_name')

In [159]:
# Fill missing concept names with the concept code (if there was no SNOMED code to be found)
terms_sorted['concept_name'] = terms_sorted.concept_name.fillna(terms_sorted.concept_code)

# Add "No Vocabulary" for vocabulary_id for these
terms_sorted['vocabulary_id'] = terms_sorted.vocabulary_id.fillna('No Vocabulary')

# Combine concept names if multiple concepts were used
terms_sorted['combined_concept'] = terms_sorted.concept_name + terms_sorted.concept_name_2.fillna('').apply(lambda x: ' - ' + x if x else '')
# + terms_sorted.time.fillna('').apply(lambda x: ' - ' + str(x) if x else '')

In [160]:
terms_sorted[['term','combined_concept']].to_csv('terms_and_concept_names.csv')

In [161]:
terms_sorted_dict = terms_sorted.set_index('term')['combined_concept'].to_dict()

### Apply Switches

In [162]:
terms_sorted[terms_sorted.direction=='switch']

,term,concept_code,concept_code_2,time,direction,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,concept_id_2,concept_name_2,domain_id_2,vocabulary_id_2,concept_class_id_2,combined_concept
241,cognitive decline,373930000,NaN,NaN,switch,4162723.0,Cognitive function finding,Observation,SNOMED,Clinical Finding,NaN,NaN,NaN,NaN,NaN,Cognitive function finding
64,Cognitive dysfunction,373930000,NaN,NaN,switch,4162723.0,Cognitive function finding,Observation,SNOMED,Clinical Finding,NaN,NaN,NaN,NaN,NaN,Cognitive function finding
213,Physical Inactivity,256235009,NaN,NaN,switch,4116678.0,Exercise,Observation,SNOMED,Observable Entity,NaN,NaN,NaN,NaN,NaN,Exercise
246,female,248153007,NaN,NaN,switch,442985.0,Male,Observation,SNOMED,Clinical Finding,NaN,NaN,NaN,NaN,NaN,Male
110,sex(female),248153007,NaN,NaN,switch,442985.0,Male,Observation,SNOMED,Clinical Finding,NaN,NaN,NaN,NaN,NaN,Male
148,female sex,248153007,NaN,NaN,switch,442985.0,Male,Observation,SNOMED,Clinical Finding,NaN,NaN,NaN,NaN,NaN,Male
172,female gender,248153007,NaN,NaN,switch,442985.0,Male,Observation,SNOMED,Clinical Finding,NaN,NaN,NaN,NaN,NaN,Male
302,reduced QoL/ADL,405152002,NaN,NaN,switch,4234649.0,Quality of life satisfaction,Observation,SNOMED,Observable Entity,NaN,NaN,NaN,NaN,NaN,Quality of life satisfaction


In [163]:
switch_terms = terms_sorted[terms_sorted.direction=='switch'].term

In [164]:
def flip_direction(direction):
    return "Decrease" if direction == "Increase" else "Increase"

In [165]:
def apply_switch_logic(row):
    # Count how many terms in `switch_terms` appear in `Exposure` or `Outcome`
    match_count = sum(term in row["Exposure"] or term in row["Outcome"] for term in switch_terms)
    
    # If there is exactly one match, flip the direction
    if match_count == 1:
        return flip_direction(row["Direction"])
    # If there are two matches (both terms are present), keep the direction unchanged
    else:
        return row["Direction"]

In [166]:
# Apply the function to the "Direction" column
df["Direction"] = df.apply(apply_switch_logic, axis=1)

### Combine mapping with original DAG data

In [167]:
df['Exposure'] = df.Exposure.replace(terms_sorted_dict)
df['Outcome'] = df.Outcome.replace(terms_sorted_dict)
df.head()

,Author,Exposure,Outcome,Direction,Strength,ID,Status
0,NaN,Diabetes mellitus,Ischemic stroke,Increase,6.0,1,Final
1,NaN,Age factor,Ischemic stroke,Increase,6.0,1,Final
2,NaN,Age factor,Diabetes mellitus,Increase,5.0,1,Final
3,NaN,Male,Ischemic stroke,Increase,6.0,1,Final
4,NaN,Essential hypertension,Ischemic stroke,Increase,6.0,1,Final


In [169]:
df.to_csv('standardized_DAG_table.csv', index=False)